# NASS Statistics by Federal Reserve Bank District

Which Federal Reserve Bank district has the most farmland? What commodities generate the most income in each district? Which district sells the most llamas? Using USDA 2022 Ag Census data and a nifty shapefile compiled by Colton Tousey of the Federal Reserve Bank of Kansas City, we can answer all of these questions, and more.

In [1]:
import polars as pl
import requests as req
import os
from dotenv import load_dotenv
import json
import time
from time import sleep
from alive_progress import alive_bar
from wakepy import keep
import polars.selectors as cs
from great_tables import GT, style, loc
import os
import json
import random

## 1. Gathering Data

First, we need an API key from the USDA to query the NASS database. This is an untracked file in the GitHub repository for this project; it needs to be independently requested from the USDA by whoever wants to run this code.

FedCounties.csv records the Federal Reserve Bank district for every county in the United States, along with state and county FIPS codes. NASS statistics also include FIPS codes.

Keep running the following block until all the counties are accounted for and it runs through the whole thing in a second or two.

Thank you for your help, Liam!

In [8]:
def pull_single(state_fips_code: str, county_code: str, year: int):
    from dotenv import load_dotenv
    load_dotenv()

    raw = req.get(
        "https://quickstats.nass.usda.gov/api/api_GET",
        params={
            "key": os.getenv("NASS_api_key"),
            "state_fips_code": state_fips_code,
            "county_code": county_code,
            "agg_level_desc": "COUNTY",
            "source_desc": "CENSUS",
            "year": year,
            "format": "json",
        },
    ).text

    try:
        content = json.loads(raw)
    except json.decoder.JSONDecodeError as e: # 403, forbidden
        if '403' in raw:
            raise e
        else:
            raise RuntimeError(f"Got something else?? {raw}")

    if "error" in content:
        print(content["error"])
        raise ValueError()

    county_df = pl.DataFrame(json.loads(raw)["data"])
    county_df = county_df.select(
        [pl.col(c) for c in sorted(county_df.columns)]
    )
    return county_df

fed_counties_df = pl.read_csv("FedCounties.csv").filter(~pl.col("STATEFP").is_in([78, 72, 69, 66, 60]))

year = 2022

for dist in range(1, 13):
    df_dist = fed_counties_df.filter(fed_counties_df["District"] == dist)

    with alive_bar(df_dist.height, title=f"District {dist}") as bar:
        for row in df_dist.iter_rows(named=True):
            state, county = str(row["STATEFP"]).zfill(2), str(row['COUNTYFP']).zfill(3)
            skip = False
            save_path = f'./data/{year}.parquet/state_fips_code={state}/county_code={county:03}/00000000.parquet'
            if os.path.exists(save_path):
                skip = True
            with open('./data/bad_combos.csv', 'r') as f:
                for line in f:
                    if f'{state},{county}' in line:
                        skip = True
                        break
            
            if skip:
                bar(skipped=True)
                continue

            try:
                county_df = pull_single(state, county, year=year)
                time.sleep(1.5)
            except json.JSONDecodeError:
                print('Error occured, lets try again later')
            except ValueError:
                with open('./data/bad_combos.csv', 'a') as f:
                    f.write(f'{state},{county}\n')
                continue

            county_df = county_df.with_columns(pl.lit(dist).alias("district"))
            if county_df.height > 50_000:
                raise ValueError('County DataFrame has max number of rows, data likely missing')
            county_df.write_parquet(f'./data/{year}.parquet', partition_by=('state_fips_code', 'county_code'))
            bar()

District 1 |█████████████████████████████▏⚠︎         | (!) 48/66 [73%] in 2:19.6 (0.34/s) 


KeyboardInterrupt: 

Now we have the unique county, state, Fed district pairs. The next step is to gather ALL 2022 Census data for every county and add a new variable to the USDA data: "District".

Note: The U.S. Virgin Islands and other territories are excluded from the USDA Ag Census, but Puerto Rico is included below.

In [11]:
year = 2018

raw = req.get(
    "https://quickstats.nass.usda.gov/api/api_GET",
    params={
        "key": os.getenv("NASS_api_key"),
        "agg_level_desc": "PUERTO RICO & OUTLYING AREAS",
        "state_name": "PUERTO RICO",
        "source_desc": "CENSUS",
        "year": year,
        "format": "json",
    },
).text

try:
    content = json.loads(raw)
except json.decoder.JSONDecodeError as e:
    print(raw)
    raise e

pr_df = pl.DataFrame(json.loads(raw)["data"])
pr_df = pr_df.select([pl.col(c) for c in sorted(pr_df.columns)])
NASS_pull_pr = pr_df.with_columns(pl.lit(2).alias("district"))
NASS_pull_pr.write_parquet(f"{year}_pr.parquet")

The final dataset is stored as a .parquet file; this is very similar to a CSV file but it takes up a fraction of the space. There are over 3 million rows in "NASS_pull.parquet; a CSV file with that many rows costs actual money to upload to GitHub.

## 2. Cleaning

Some values in the final dataset are not actual values, so we need to filter these rows out. Then, we can aggregate our data to get rid of extraneous information, which at this point is any and all columns excluding "short_desc".

In [59]:
df_big = pl.read_parquet("2017.parquet")
df_pr = pl.read_parquet("2018_pr.parquet")

# df.filter(expr1, expr2) # and
df_pr = df_pr.filter((pl.col("region_desc") == "PUERTO RICO") & (pl.col("domain_desc") == "TOTAL"))
dfs = [df_big, df_pr]
df = pl.concat(dfs)
df = df.filter(
    (~pl.col("Value").str.contains(r"\(D\)|\(Z\)")) & ((pl.col("domain_desc") == "TOTAL"))
)
df = df.with_columns(pl.col("Value").str.replace_all(",", "").cast(pl.Float64))

# want to know if there are multiple of the same short_desc entries per county...
# want to know if filtering by domain_desc == "TOTAL" ensures that there are ONLY unique short_descs for each county...
# grouped_df = df.group_by(['state_fips_code', 'county_code', 'short_desc']).agg(
#     pl.len().alias('count')
# )

# # Now find which state_fips_code and county_code combinations have duplicated short_desc values
# # We group again by state_fips_code and county_code, and filter where any short_desc appears more than once
# result = grouped_df.group_by(['state_fips_code', 'county_code']).agg([
#     pl.len().alias('unique_short_desc_count'),
#     (pl.col('count') > 1).any().alias('has_duplicate_short_desc')
# ]).filter(
#     pl.col('has_duplicate_short_desc') == True
# )

# print(result)
# print(result["unique_short_desc_count"].sum())

# thank you Claude

district_dfs = []

for dist in df.partition_by("district"):
    district_df = dist.group_by("short_desc").agg(
        [
            pl.when(pl.col("short_desc").str.contains("PCT") | pl.col("short_desc").str.contains("/ OPERATION") | pl.col("short_desc").str.contains("/ ACRE"))
            .then(pl.col("Value").mean())
            .otherwise(pl.col("Value").sum())
            .alias("district_total"),
            pl.mean("district").cast(pl.Int32),
        ]
    )
    district_dfs.append(district_df)

df = pl.concat(district_dfs)
print(df)

shape: (20_769, 3)
┌─────────────────────────────────┬─────────────────────────────────┬──────────┐
│ short_desc                      ┆ district_total                  ┆ district │
│ ---                             ┆ ---                             ┆ ---      │
│ str                             ┆ list[f64]                       ┆ i32      │
╞═════════════════════════════════╪═════════════════════════════════╪══════════╡
│ CAMELINA - PRODUCTION, MEASURE… ┆ [3420.0]                        ┆ 6        │
│ TREE NUT TOTALS - ACRES BEARIN… ┆ [107049.0, 107049.0, … 107049.… ┆ 6        │
│ PRODUCERS, PRINCIPAL, DAYS WOR… ┆ [13872.0, 13872.0, … 13872.0]   ┆ 6        │
│ CRUSTACEANS - OPERATIONS WITH … ┆ [657.0, 657.0, … 657.0]         ┆ 6        │
│ ASPARAGUS, FRESH MARKET - ACRE… ┆ [11.0, 11.0, … 11.0]            ┆ 6        │
│ …                               ┆ …                               ┆ …        │
│ TURNIPS, FRESH MARKET - OPERAT… ┆ [173.0, 173.0, … 173.0]         ┆ 9        │
│ CHICKEN

Note: We take the median percentages (robust to outliers) across all counties in the dataset. The interpretation of these values is not super intuitive. Each mean percent is the "average percent ___ for all counties in the district", not the percent ___ for the district. We also make sure that domain_desc = "TOTAL" or else we double-count some values.

Also, the conditional aggregation creates a dataframe where "District_Total" is actually a column of lists. We resolve this in step 3.

## 3. Analyzing

***RUN FROM HERE***

To filter through the data and find commodities that we want to know more about, we can use a keyword search approach applied to the short description of the data item. Some examples are presented below:

In [60]:
districts = range(1, 13)

# income and expenses
# keyword_pairs = [
#     # expenses
#     (
#         ["expense totals, operating", "measured in \$"],
#         ["operation", "landlord"],
#     ),
#     (
#         ["taxes, property, real estate", "non-real estate", "measured in \$"], 
#         ["only", "operations"]),
#     (
#         ["rent, cash", "\$"], 
#         ["only", "operation"]
#     ),
#     (
#         ["interest", "expense", "\$"], 
#         ["pct", "/ operation", "operations", "for", "real"]
#     ),
#     (
#         ["depreciation", "expense", "\$"], 
#         ["operations"]
#     ),
#     #income
#     (
#         ["commodity totals", "sales", "\$"], 
#         ["operation", "marketed", "direct", "landlord", "organic", "retail", "value"]
#     ),
#     (
#         ["by-products", "receipts", "\$"], 
#         ["operations"]
#     ),
#     (
#         ["govt programs", "receipts", "\$"], 
#         ["operation", "conservation"]
#     ),
#     (
#         ["income, farm-related", "receipts", "\$"], 
#         ["operation", "associated", "services", "tourism", "payments", "programs", "products", "other", "dividends"]
#     ),
#     (
#         ["income, net cash farm", "operations", "net", "\$"], 
#         ["/ operation"]
#     ),
# ]

# income
# keyword_pairs = [
#     (
#         ["income", "receipts, measured in \$"], 
#         ["nada"]
#     ),
# ]

# ag land
keyword_pairs = [
    (
        ["ag land, \(excl c", "acres"], 
        ["nada"]
    ),
    (
        ["ag land, \(excl c", "cuerdas"], 
        ["nada"]
    ),
    (
        ["ag land, cropland"], 
        ["nada"]
    ),
    (
        ["ag land, pastureland", "excl", "crop", "wood", "acres"], 
        ["nada"]
    ),
    (
        ["ag land, pastureland", "excl", "crop", "wood", "cuerdas"], 
        ["nada"]
    ),
    (
        ["ag land, woodland", "acres"], 
        ["excl"]
    ),
    (
        ["ag land, woodland", "cuerdas"], 
        ["nada"]
    ),
    (
        ["ag land, incl buildings"], 
        ["operations"]
    ),
    (
        ["land area, incl non-ag"], 
        ["crop"]
    ),
]
# keyword_pairs = [
#     (
#         ["ag land", "acres"], 
#         ["nada"]
#     ),
# ]


keyword_list = [pair[0] for pair in keyword_pairs]
excl_keyword_list = [pair[1] for pair in keyword_pairs]

# # TOP COMMODITIES
# commodity_keywords = ["sales, measured in \$"]
# keyword_list.append(commodity_keywords)
# commodity_excl_keywords = ["totals"]
# excl_keyword_list.append(commodity_excl_keywords)

dfs = []

for incl, excl in zip(keyword_list, excl_keyword_list):
    custom = df.filter(
        [pl.col("short_desc").str.to_lowercase().str.contains(k.lower()) for k in incl],
        *[
            ~pl.col("short_desc").str.to_lowercase().str.contains(exk.lower())
            for exk in excl
        ],
        pl.col("district").is_in(districts),
    )
    dfs.append(custom)

custom = pl.concat(dfs)
custom = custom.with_columns(pl.col("district_total").list.unique().list.first())
custom = custom.unique(maintain_order=True)
custom.write_parquet("custom_df.parquet")

If we know exactly which data items we would like included in a final table, then we can move on to 4. If some extra analysis needs doing, then go to step 3a first.

### 3a. More Analyzing

If there are some secondary characteristics we want more information on, such as which commodities generate the most cash sales in each district, then some more work needs to be done before a dataframe will be ready for final formatting. Below we find the top 10 highest value commodities in each district, per our earlier keyword search.

In [ ]:
df_2 = pl.read_parquet("custom_df.parquet")

district_dfs = []

for dist in df_2.partition_by("district"):
    district_df = dist.sort("district_total", descending=True).head(20)
    district_dfs.append(district_df)

df_2 = pl.concat(district_dfs)
print(df_2)

df_2.write_parquet("custom_df.parquet")

shape: (134, 3)
┌─────────────────────────────────┬────────────────┬──────────┐
│ short_desc                      ┆ District_Total ┆ District │
│ ---                             ┆ ---            ┆ ---      │
│ str                             ┆ f64            ┆ i32      │
╞═════════════════════════════════╪════════════════╪══════════╡
│ INCOME, FARM-RELATED - RECEIPT… ┆ 3.26761e8      ┆ 1        │
│ INCOME, FARM-RELATED, OTHER - … ┆ 1.49519e8      ┆ 1        │
│ INCOME, FARM-RELATED, AG TOURI… ┆ 4.9575e7       ┆ 1        │
│ INCOME, FARM-RELATED, FOREST P… ┆ 4.264e7        ┆ 1        │
│ INCOME, FARM-RELATED, AG SERVI… ┆ 2.7924e7       ┆ 1        │
│ …                               ┆ …              ┆ …        │
│ INCOME, FARM-RELATED, AG TOURI… ┆ 1.93228e8      ┆ 12       │
│ INCOME, FARM-RELATED, FOREST P… ┆ 1.44229e8      ┆ 12       │
│ INCOME, FARM-RELATED, GOVT PRO… ┆ 1.5043e7       ┆ 12       │
│ INCOME, FARM-RELATED - RECEIPT… ┆ 41680.0        ┆ 12       │
│ INCOME, FARM-RELATED, 

## 4. Table Formatting

In [61]:
dict = {
    "short_desc": "Description",
    "1": "Boston",
    "2": "New York",
    "3": "Philadelphia",
    "4": "Cleveland",
    "5": "Richmond",
    "6": "Atlanta",
    "7": "Chicago",
    "8": "St. Louis",
    "9": "Minneapolis",
    "10": "Kansas City",
    "11": "Dallas",
    "12": "San Francisco",
}

df_3 = pl.read_parquet("custom_df.parquet")

df_3 = df_3.pivot("district", values=cs.starts_with("district_total"))

df_3 = df_3.rename(dict)
df_3 = df_3.with_columns(pl.col("Description").str.to_titlecase())
df_3.write_csv("custom_table.csv")
print(df_3)

# # refer to NASS for units
gt_df = GT(df_3)

dist_cols = [
    "Boston",
    "New York",
    "Philadelphia",
    "Cleveland",
    "Richmond",
    "Atlanta",
    "Chicago",
    "St. Louis",
    "Minneapolis",
    "Kansas City",
    "Dallas",
    "San Francisco",
]

gt_df = gt_df.tab_spanner(label="district", columns=dist_cols).tab_style(
    style=style.text(size="9px", font="Helvetica"),
    locations=loc.body(columns="Description"),
)

gt_df

shape: (46, 13)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Descripti ┆ Atlanta   ┆ San       ┆ St. Louis ┆ … ┆ Cleveland ┆ Dallas    ┆ Richmond  ┆ Minneapo │
│ on        ┆ ---       ┆ Francisco ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ lis      │
│ ---       ┆ f64       ┆ ---       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ ---      │
│ str       ┆           ┆ f64       ┆           ┆   ┆           ┆           ┆           ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Ag Land,  ┆ 2.922051e ┆ 4.575026e ┆ 2.45318e6 ┆ … ┆ 986845.0  ┆ 2.584367e ┆ 1.535344e ┆ 4.63295e │
│ (Excl     ┆ 6         ┆ 6         ┆           ┆   ┆           ┆ 6         ┆ 6         ┆ 6        │
│ Cropland  ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ & Past…   ┆           ┆           ┆           ┆   ┆           ┆          

GT(_tbl_data=shape: (46, 13)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Descripti ┆ Atlanta   ┆ San       ┆ St. Louis ┆ … ┆ Cleveland ┆ Dallas    ┆ Richmond  ┆ Minneapo │
│ on        ┆ ---       ┆ Francisco ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ lis      │
│ ---       ┆ f64       ┆ ---       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ ---      │
│ str       ┆           ┆ f64       ┆           ┆   ┆           ┆           ┆           ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Ag Land,  ┆ 2.922051e ┆ 4.575026e ┆ 2.45318e6 ┆ … ┆ 986845.0  ┆ 2.584367e ┆ 1.535344e ┆ 4.63295e │
│ (Excl     ┆ 6         ┆ 6         ┆           ┆   ┆           ┆ 6         ┆ 6         ┆ 6        │
│ Cropland  ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ & Past…   ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ Ag Land,  ┆ null      ┆ null      ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null     │
│ (Excl     ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ Cropland  ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ & Past…   ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ Ag Land,  ┆ 117325.0  ┆ 130184.0  ┆ 136041.0  ┆ … ┆ 90853.0   ┆ 105864.0  ┆ 96872.0   ┆ 119004.0 │
│ Cropland, ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ Harvested ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ -…        ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ Ag Land,  ┆ 261842.0  ┆ 377364.0  ┆ 286701.0  ┆ … ┆ 95704.0   ┆ 2.346172e ┆ 156060.0  ┆ 3.16515e │
│ Cropland, ┆           ┆           ┆           ┆   ┆           ┆ 6         ┆           ┆ 6        │
│ (Excl     ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ Harve…    ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ Ag Land,  ┆ 1.70438e6 ┆ 4.17378e6 ┆ 2.597794e ┆ … ┆ 831139.0  ┆ 5.272e6   ┆ 1.043787e ┆ 6.962173 │
│ Cropland, ┆           ┆           ┆ 6         ┆   ┆           ┆           ┆ 6         ┆ e6       │
│ (Excl     ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ Harve…    ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ …         ┆ …         ┆ …         ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …        │
│ Ag Land,  ┆ null      ┆ null      ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null     │
│ Woodland, ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ (Excl     ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ Pastu…    ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ Ag Land,  ┆ 1.7542e11 ┆ 4.0023e11 ┆ 2.3419e11 ┆ … ┆ 1.1144e11 ┆ 2.6449e11 ┆ 1.1697e11 ┆ 3.4867e1 │
│ Incl      ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 1        │
│ Buildings ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ - Asse…   ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ Ag Land,  ┆ 4468.5650 ┆ 6589.5819 ┆ 3643.1065 ┆ … ┆ 4899.0887 ┆ 2340.8518 ┆ 5781.2879 ┆ 3006.174 │
│ Incl      ┆ 22        ┆ 67        ┆ 09        ┆   ┆ 57        ┆ 52        ┆ 75        ┆ 917      │
│ Buildings ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ - Asse…   ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ Ag Land,  ┆ 849598.03 ┆ 1.7786

From here, I think a good amount of hard-coding is needed for table formatting; districts will have different top-production commodities, so how do we want to display that information? It's tougher to decide than when you are comparing particular commodity classes across districts...